In [34]:
from pymongo import MongoClient
import pandas as pd

In [35]:
client = MongoClient(port=27017)
db = client.trip_advisor
collection = db.reviews
df = pd.DataFrame(list(collection.find()))

In [36]:
df.describe()

,_id,poi_name,title,date,review_rating,text,date_of_visit,reviewer
count,530,530,530,530,530,530,530,530
unique,530,15,510,439,5,529,96,497
top,"{'id': '620261466', 'poi_location_id': '116439...",Koukos,Best coffee in Thessaloniki,"March 17, 2015",50,"I went there to get a salad, staff really trie...",July 2016,"{'name': 'REDMAN', 'handle': 'REDMAN5314', 'lo..."
freq,1,86,5,5,402,2,17,4


In [37]:
print(f"Missing values are present: {df.isnull().sum().any()}")

Missing values are present: False
